## IBM Coursera Data Science Capstone Project Week 3

## Question 1

### First import the necessary libraries

In [1]:
# used to make tab autocomplete work faster 
%config Completer.use_jedi = False

In [2]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
url

'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

**Get the html contents in string format**

In [13]:
data = requests.get(url).text
# data

**Convert data to a Soup Object**

In [12]:
soup = BeautifulSoup(data, 'html.parser')
# soup

In [7]:
table = soup.find('table')
table_contents = []
for i,row in enumerate(table.findAll('td')):
    cell = {}
    if 'not assigned' in row.span.text.lower():
        pass
    else:
#         df_torneigh.iloc[i,'PostalCode'] = row.text[:3]
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = row.span.text.split('(')[0]
        cell['Neighborhood'] = row.span.text.split('(')[1].replace(' /',',').strip(')')
        table_contents.append(cell)
df = pd.DataFrame(table_contents)

In [8]:
df[df['Borough'].str.len() > 20]

,PostalCode,Borough,Neighborhood
35,M4J,East YorkEast Toronto,The Danforth East
76,M7R,MississaugaCanada Post Gateway Processing Centre,Enclave of L4W
92,M5W,Downtown TorontoStn A PO Boxes25 The Esplanade,Enclave of M5E
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L


In [9]:
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [11]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


**Shape of the dataframe**

In [84]:
df.shape

(103, 3)